# 노이즈 포함 WAV 제작

## 1. 라이브러리 설치

In [3]:
pip install librosa scipy numpy soundfile

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# 백색 소음, 핑크 노이즈 추가 작업

- 백색 잡음 (White Noise): 주파수 대역에 관계없이 일정한 파워를 가진 잡음입니다.
- 핑크 노이즈 (Pink Noise): 주파수가 높아질수록 파워가 감소하는 잡음으로, 실제 환경에서 자주 발생하는 소리입니다.

In [8]:
import os
import numpy as np
import scipy.signal as signal
import soundfile as sf

# 핑크 노이즈 생성 함수
def generate_pink_noise(length, sr):
    # 백색 잡음 생성
    white_noise = np.random.normal(0, 1, length)
    # 로우패스 필터 생성 (핑크 노이즈 필터)
    b, a = signal.butter(1, 0.02, btype='low')  
    pink_noise = signal.filtfilt(b, a, white_noise)  # 필터링
    pink_noise = pink_noise / np.max(np.abs(pink_noise))  # 정규화
    return pink_noise

# 백색 잡음 생성 함수
def generate_white_noise(length):
    return np.random.normal(0, 1, length)

# 원하는 SNR 수준으로 노이즈 추가
def add_noise_dynamic(mfcc, target_snr_db=20):
    signal_power = np.mean(mfcc ** 2)
    noise_power = signal_power / (10 ** (target_snr_db / 10))
    noise = np.random.normal(0, np.sqrt(noise_power), mfcc.shape)
    return mfcc + noise

# WAV 파일에 핑크 노이즈 또는 백색 잡음 추가
def add_noise_to_audio(file_path, noise_type='pink', target_snr_db=20, output_file='output_with_noise.wav'):
    # 원본 오디오 파일 로드
    audio, sr = sf.read(file_path)

    # 노이즈 길이 설정
    length = len(audio)
    
    # 선택한 노이즈 타입에 따라 노이즈 생성
    if noise_type == 'pink':
        noise_signal = generate_pink_noise(length, sr)
    elif noise_type == 'white':
        noise_signal = generate_white_noise(length)
    else:
        raise ValueError("Noise type must be either 'pink' or 'white'")

    # 노이즈 추가 (원하는 SNR 수준으로 조정)
    noisy_audio = add_noise_dynamic(audio, target_snr_db)
    
    # 현재 작업 디렉토리에서 파일 저장
    current_dir = os.getcwd()  # 현재 작업 디렉토리
    output_file_path = os.path.join(current_dir, output_file)  # 경로 설정

    # 노이즈가 포함된 파일 저장
    sf.write(output_file_path, noisy_audio, sr)
    print(f"노이즈가 포함된 파일이 저장되었습니다: {output_file_path}")

# 파일 경로 지정
file_path_car_horn = "/home/ubuntu/data/test_data/raw_data_test/1.Car/1.horn_of_car/1.car_horn_10_1.wav"
file_path_motorcycle_sound = "/home/ubuntu/data/test_data/raw_data_test/2.Motorcycle/5.driving_sound_of_motorcycle/2.motorcycle_driving_sound_1860_1.wav"

# 노이즈 추가 (핑크 노이즈, 백색 잡음, 그리고 SNR 설정)
add_noise_to_audio(file_path_car_horn, noise_type='pink', target_snr_db=15, output_file="car_horn_with_pink_noise.wav")
add_noise_to_audio(file_path_motorcycle_sound, noise_type='white', target_snr_db=20, output_file="motorcycle_sound_with_white_noise.wav")

노이즈가 포함된 파일이 저장되었습니다: /home/lab06/git/noise-data-platform/JW/car_horn_with_pink_noise.wav
노이즈가 포함된 파일이 저장되었습니다: /home/lab06/git/noise-data-platform/JW/motorcycle_sound_with_white_noise.wav


In [12]:
import numpy as np
import librosa

def calculate_snr(signal, noisy_signal):
    # Signal power (원본 신호의 평균 제곱값)
    signal_power = np.mean(signal ** 2)
    
    # Noise power (노이즈의 평균 제곱값)
    noise = noisy_signal - signal  # 노이즈는 원본 신호와 노이즈 신호의 차이
    noise_power = np.mean(noise ** 2)
    
    # 노이즈가 너무 적으면 noise_power를 1e-10으로 설정
    if noise_power < 1e-10:
        noise_power = 1e-10
    
    # SNR 계산 (dB 단위)
    snr = 10 * np.log10(signal_power / noise_power)
    return snr, signal_power, noise_power

# 예시: 원본 신호와 노이즈가 포함된 신호 경로
file_path_original = '/home/ubuntu/data/test_data/raw_data_test/1.Car/1.horn_of_car/1.car_horn_10_1.wav'
file_path_noisy = 'car_horn_with_pink_noise.wav'

# 원본 신호와 노이즈가 포함된 신호 불러오기
signal, sr = librosa.load(file_path_original, sr=None)
noisy_signal, _ = librosa.load(file_path_noisy, sr=None)

# 원본 SNR 계산
snr_original, signal_power_original, noise_power_original = calculate_snr(signal, signal)
# 노이즈가 추가된 SNR 계산
snr_noisy, signal_power_noisy, noise_power_noisy = calculate_snr(signal, noisy_signal)

# SNR 차이 계산
snr_difference = snr_original - snr_noisy

# 결과 출력
print(f"Original SNR: {snr_original:.2f} dB")
print(f"Noisy SNR: {snr_noisy:.2f} dB")
print(f"SNR Difference: {snr_difference:.2f} dB")


Original SNR: 73.36 dB
Noisy SNR: 14.99 dB
SNR Difference: 58.37 dB


In [7]:
import numpy as np
import librosa

def calculate_snr(signal, noisy_signal):
    # Signal power (원본 신호의 평균 제곱값)
    signal_power = np.mean(signal ** 2)
    
    # Noise power (노이즈의 평균 제곱값)
    noise = noisy_signal - signal  # 노이즈는 원본 신호와 노이즈 신호의 차이
    noise_power = np.mean(noise ** 2)
    
    # 노이즈가 너무 적으면 noise_power를 1e-10으로 설정
    if noise_power < 1e-10:
        noise_power = 1e-10
    
    # SNR 계산 (dB 단위)
    snr = 10 * np.log10(signal_power / noise_power)
    return snr, signal_power, noise_power

# 예시: 원본 신호와 노이즈가 포함된 신호 경로
file_path_original = '/home/ubuntu/data/test_data/raw_data_test/2.Motorcycle/5.driving_sound_of_motorcycle/2.motorcycle_driving_sound_1860_1.wav'
file_path_noisy = 'motorcycle_sound_with_white_noise.wav'

# 원본 신호와 노이즈가 포함된 신호 불러오기
signal, sr = librosa.load(file_path_original, sr=None)
noisy_signal, _ = librosa.load(file_path_noisy, sr=None)

# 원본 SNR 계산
snr_original, signal_power_original, noise_power_original = calculate_snr(signal, signal)
# 노이즈가 추가된 SNR 계산
snr_noisy, signal_power_noisy, noise_power_noisy = calculate_snr(signal, noisy_signal)

# SNR 차이 계산
snr_difference = snr_original - snr_noisy

# 결과 출력
print(f"Original SNR: {snr_original:.2f} dB")
print(f"Noisy SNR: {snr_noisy:.2f} dB")
print(f"SNR Difference: {snr_difference:.2f} dB")


Original SNR: 79.73 dB
Noisy SNR: 23.06 dB
SNR Difference: 56.67 dB


## SNR 크기별 파일 생성

In [36]:
import numpy as np
import librosa
import soundfile as sf

# 원본 SNR을 계산하는 함수 (inf 문제 해결)
def calculate_snr(signal, noise):
    signal_power = np.mean(signal ** 2)
    noise_power = np.mean(noise ** 2)
    
    # 신호가 0이면 SNR을 무한대 대신 매우 큰 값(100dB)으로 설정
    if signal_power == 0:
        return 100
    
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

# 원하는 SNR 수준으로 노이즈 추가 (inf 문제 방지)
def add_noise_dynamic(mfcc, target_snr_db=20):
    # 원본 신호에 아주 작은 노이즈 추가하여 inf 방지
    mfcc = mfcc + np.random.normal(0, 1e-6, mfcc.shape)
    
    # 신호와 노이즈 파워 계산
    signal_power = np.mean(mfcc ** 2)
    noise_power = signal_power / (10 ** (target_snr_db / 10))
    
    # 노이즈 생성 및 추가
    noise = np.random.normal(0, np.sqrt(noise_power), mfcc.shape)
    noisy_mfcc = mfcc + noise
    
    # SNR 계산
    original_snr = calculate_snr(mfcc, np.zeros_like(mfcc))
    noisy_snr = calculate_snr(mfcc, noise)
    snr_difference = original_snr - noisy_snr
    
    # 디버깅용 출력
    print(f"Original SNR: {original_snr:.2f} dB")
    print(f"Noisy SNR: {noisy_snr:.2f} dB")
    print(f"SNR Difference: {snr_difference:.2f} dB")
    
    return noisy_mfcc, noise

# WAV 파일을 불러오고 노이즈 추가 후 저장
def process_wav_file(input_path, output_path, target_snr_db=20):
    # 오디오 파일 로드
    signal, sr = librosa.load(input_path, sr=None)
    
    # 노이즈 추가
    noisy_signal, noise = add_noise_dynamic(signal, target_snr_db)
    
    # 새로운 파일로 저장
    sf.write(output_path, noisy_signal, sr)
    print(f"Processed file saved: {output_path}")

# 원본 파일 경로
input_paths = [
    "/home/ubuntu/data/test_data/raw_data_test/1.Car/1.horn_of_car/1.car_horn_10_1.wav",
    "/home/ubuntu/data/test_data/raw_data_test/2.Motorcycle/5.driving_sound_of_motorcycle/2.motorcycle_driving_sound_1860_1.wav"
]

# 저장할 경로
output_paths = [
    "1.car_horn_10_1_noisy.wav",
    "2.motorcycle_driving_sound_1860_1_noisy.wav"
]

# 다양한 SNR 수준 테스트 (20dB, 15dB, 10dB)
target_snr_levels = [20, 15, 10]

for snr_db in target_snr_levels:
    print(f"\n🔹 Testing with target SNR: {snr_db} dB")
    for input_path, output_path in zip(input_paths, output_paths):
        modified_output_path = output_path.replace(".wav", f"_{snr_db}dB.wav")
        process_wav_file(input_path, modified_output_path, target_snr_db=snr_db)



🔹 Testing with target SNR: 20 dB
Original SNR: inf dB
Noisy SNR: 19.98 dB
SNR Difference: inf dB
Processed file saved: 1.car_horn_10_1_noisy_20dB.wav
Original SNR: inf dB
Noisy SNR: 20.05 dB
SNR Difference: inf dB
Processed file saved: 2.motorcycle_driving_sound_1860_1_noisy_20dB.wav

🔹 Testing with target SNR: 15 dB
Original SNR: inf dB
Noisy SNR: 15.01 dB
SNR Difference: inf dB
Processed file saved: 1.car_horn_10_1_noisy_15dB.wav
Original SNR: inf dB
Noisy SNR: 14.95 dB
SNR Difference: inf dB
Processed file saved: 2.motorcycle_driving_sound_1860_1_noisy_15dB.wav

🔹 Testing with target SNR: 10 dB
Original SNR: inf dB
Noisy SNR: 10.01 dB
SNR Difference: inf dB
Processed file saved: 1.car_horn_10_1_noisy_10dB.wav
Original SNR: inf dB
Noisy SNR: 10.00 dB
SNR Difference: inf dB
Processed file saved: 2.motorcycle_driving_sound_1860_1_noisy_10dB.wav


/tmp/ipykernel_18261/4038009251.py:14: RuntimeWarning: divide by zero encountered in scalar divide
  snr = 10 * np.log10(signal_power / noise_power)
